In [2]:
import os
import tensorflow as tf
import cv2 as cv
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Lambda,Dense,Flatten, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.callbacks import EarlyStopping


from keras.utils.vis_utils import plot_model

In [3]:
def basic_preProcess(img):
    img = resize_image(img)
    return img

def resize_image(img):
  image_size = (224,224)
  img = cv.resize(img, image_size,interpolation = cv.INTER_LINEAR)
  return img

In [5]:
def color_preProcess(img):
  #insert methods
  # img = to_colorhsv(img)
  # img = to_CIELAB(img)
  # img = to_CIEXYZ(img)
  # img = quantize_image(img,8)
  # img = to_RGB(img)


  return img

def to_colorhsv(img):

  hue_shift = 90
  saturation_scale = 1
  value_scale = 1

  img[..., 1] = np.clip(img[..., 1] * saturation_scale, 0, 255)
  img[..., 2] = np.clip(img[..., 2] * value_scale, 0, 255)
  img[..., 2] = cv.equalizeHist(img[..., 2])

  img = cv.cvtColor(img, cv.COLOR_HSV2BGR)

  return img

def to_CIELAB(img):
  img = cv.cvtColor(img,cv.COLOR_BGR2Lab)
  return img

def to_CIEXYZ(img):
  img = cv.cvtColor(img,cv.COLOR_BGR2XYZ)
  return img

def to_RGB(img):
  img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
  return img

def quantize_image(image, n_colors = 4):
    lab_image = cv.cvtColor(image, cv.COLOR_BGR2Lab)
    pixel_values = lab_image.reshape((-1, 3))
    kmeans = KMeans(n_clusters=n_colors, random_state=42)
    kmeans.fit(pixel_values)
    centers = kmeans.cluster_centers_
    quantized_pixels = centers[kmeans.predict(pixel_values)]
    quantized_image = quantized_pixels.reshape(lab_image.shape).astype(np.uint8)
    quantized_bgr_image = cv.cvtColor(quantized_image, cv.COLOR_Lab2RGB)

    return quantized_bgr_image


In [ ]:
def category_preProcess(img):
  # img = to_grayhsv(img)
  img = gamma_correct_HSV(img)
  return img

def to_grayhsv(img):

  saturation_scale = 0
  value_scale = 0.1

  img[..., 1] = np.clip(img[..., 1] * saturation_scale, 0, 255)
  img[..., 2] = np.clip(img[..., 2] * value_scale, 0, 255)
  img[..., 2] = cv.equalizeHist(img[..., 2])


  img = cv.cvtColor(img, cv.COLOR_HSV2BGR)

  # img = cv.GaussianBlur(img,(5,5),0)

  return img

def gamma_correct_HSV(img):

  gamma = 2.0

  # hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
  H, S, V = cv.split(img)

  lookup_table = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in np.arange(0, 256)]).astype("uint8")

  V_corrected = cv.LUT(V, lookup_table)
  corrected_hsv = cv.merge([H, S, V_corrected])
  corrected_image = cv.cvtColor(corrected_hsv, cv.COLOR_HSV2BGR)

  return corrected_image